<a href="https://colab.research.google.com/github/DEli-26/DS_Practicum/blob/main/06_megaline_tarif/deli_proj_pract_06_megaline_tarif_v2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size=6><b>**Я.Практикум: Проект №6**</b></font>
    
<font size=6><b>**Мегалайн. Тариф**</b></font>

***

# Постановка задачи

**Заказчик:** Оператор мобильной связи «Мегалайн».

**Цель:** построить модель для задачи классификации, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра». Accuracy модели должно быть не менее 0,75.

**Задачи:** 
1. получить данные и изучить их структуру;
1. разделить данные на обучающую, валидационную и тестовую выборки;
1. подобрать модель с лучшей достоверностью;
1. проверить модель на тестовой выборке;
1. проверить модель на адекватность.

**Исходные данные:** прошедшие предобработку данные о поведении клиентов, которые уже перешли на тарифы «Смарт» и «Ультра». 
Каждый объект — это информация о поведении одного пользователя за месяц. 

# Обзор данных

Загрузим библиотеки, необходимые для выполнения проекта

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

# Темная тема графиков
plt.style.use('dark_background')

# Отключаем предупреждения
import warnings

warnings.filterwarnings("ignore")

-- Здесь был код загрузки данных с Google Drive --

Откроем первые 5 строк, а также общую информацию о таблице.

In [ ]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
df.duplicated().sum()

0

Таблица содержит следующие столбцы:
* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

В исследуемой таблице содержится информация о количестве звонков за месяц, израсходованных пользователями минут, сообщений и интернет-трафика, а также о типе тарифа.
Пропуски и явные дубликаты отсутствуют. Тип данных соответствует их описанию.

Таким образом, можно переходить к разбиению таблицы на выборки и обучению моделей на их основе.

# Разделение исходных данных 

Оценим количество целевых показателей в исходной таблице.

In [ ]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

По полученным значениям хорошо видно, что данные в столбце с ключевым показателем несбалансированы. Значение "0" встречается в 69% случаев, а "1" - в 31%. В этой связи возможно недообучение моделей для анализа критерия со значением "1".

Разделим таблицу на обучающую, валидационную и тестовую выборки в пропорциях 3:1:1 соответственно. Учитывая несбалансированность данных, используем опцию `stratify` для колонки `is_ultra`.

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> 
Внес правки здесь по результатам ознакомления с любезно предоставленными тобой ссылками.
</div>

In [ ]:
df_train, df_valid_test = train_test_split(df,
                                           test_size=.4,
                                           random_state=26,
                                           stratify=df['is_ultra'])
df_valid, df_test = train_test_split(df_valid_test,
                                     test_size=.5,
                                     random_state=26,
                                     stratify=df_valid_test['is_ultra'])

In [ ]:
len(df_train)

1928

In [ ]:
len(df_valid)

643

In [ ]:
len(df_test)

643

Разделение прошло успешно, данные не потерялись. Выделим из полученной таблицы целевые признаки - тарифы и отделим их от остальных данных. Для этого создадим функцию `separate_target_features`.

In [ ]:
def separate_target_features(data):
    return data['is_ultra'], data.drop('is_ultra', axis=1)

In [ ]:
df_train_target, df_train_features = separate_target_features(df_train)
df_valid_target, df_valid_features = separate_target_features(df_valid)
df_test_target, df_test_features = separate_target_features(df_test)

In [ ]:
df_train_target.head()

1764    1
1559    1
2655    1
1007    0
2677    0
Name: is_ultra, dtype: int64

In [ ]:
df_train_features.head()

,calls,minutes,messages,mb_used
1764,6.0,29.77,0.0,365.09
1559,106.0,781.37,26.0,37962.31
2655,106.0,735.90,55.0,32050.45
1007,73.0,550.58,25.0,18322.63
2677,25.0,193.57,0.0,5529.38


In [ ]:
df_train_target.value_counts(normalize=True)

0    0.693465
1    0.306535
Name: is_ultra, dtype: float64

Исходная таблица разделена на 6, по две для каждого типа выборки: обучающей, валидационной и тестовой.
Можно переходить к выбору и обучению моделей.

# Выбор моделей

Решаемая задача - сводится к определению классификационной модели. Рассмотрим следующие типы моделей:
* дерево решений;
* случайный лес;
* логистическая регрессия.

Определим перечисленные модели, обучим их на обучающих выборках и подберем гиперпараметры, обеспечивающие максимизацию достоверности (accuracy). 
Последнюю задачу решим при помощи функции RandomizedSearchCV.

In [ ]:
#Пространство параметров дерева решений
max_depth = [int(x) for x in np.linspace(2, 15, num=14)]
max_features = ['log2', 'sqrt', None]
min_samples_split = [int(x) for x in np.linspace(2, 15, num=14)]
min_samples_leaf = [int(x) for x in np.linspace(2, 10, num=9)]
space_tree = {
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf
}

#Пространство параметров случайного леса
n_estimators = [int(x) for x in np.linspace(start=10, stop=100, num=19)]
space_rf = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf
}

#Пространство параметров логистической регрессии
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
penalty = ['l1', 'l2', 'elasticnet', 'none']
max_iter = [int(x) for x in np.linspace(start=10, stop=150, num=15)]
space_logreg = {'solver': solver, 'penalty': penalty, 'max_iter': max_iter}

In [ ]:
model_tree = RandomizedSearchCV(
    estimator=DecisionTreeClassifier(random_state=26),
    param_distributions=space_tree,
    cv=5,
    n_iter=200,
    random_state=26)
model_tree.fit(df_train_features, df_train_target)
print('Лучшая модель дерева решения определена со следующими параметрами:',
      model_tree.best_params_)
print('Достоверность модели дерева решения на валидационной выборке составила',
      round(model_tree.score(df_valid_features, df_valid_target), 3))

Лучшая модель дерева решения определена со следующими параметрами: {'min_samples_split': 14, 'min_samples_leaf': 9, 'max_features': None, 'max_depth': 6}
Достоверность модели дерева решения на валидационной выборке составила 0.812


In [ ]:
model_rf = RandomizedSearchCV(estimator=RandomForestClassifier(random_state=26,
                                                               n_jobs=-1),
                              param_distributions=space_rf,
                              n_jobs=-1,
                              cv=5,
                              n_iter=200,
                              random_state=26)
model_rf.fit(df_train_features, df_train_target)
print('Лучшая модель случайного леса определена со следующими параметрами:',
      model_rf.best_params_)
print(
    'Достоверность модели случайного леса на валидационной выборке составила',
    round(model_rf.score(df_valid_features, df_valid_target), 3))

Лучшая модель случайного леса определена со следующими параметрами: {'n_estimators': 85, 'min_samples_split': 15, 'min_samples_leaf': 4, 'max_features': None, 'max_depth': 10}
Достоверность модели случайного леса на валидационной выборке составила 0.818


In [ ]:
model_logreg = RandomizedSearchCV(estimator=LogisticRegression(random_state=26,
                                                               n_jobs=-1),
                                  param_distributions=space_logreg,
                                  n_jobs=-1,
                                  cv=5,
                                  n_iter=200,
                                  random_state=26)
model_logreg.fit(df_train_features, df_train_target)
print(
    'Лучшая модель логистической регрессии определена со следующими параметрами:',
    model_logreg.best_params_)
print(
    'Достоверность модели логистической регрессии на валидационной выборке составила',
    round(model_logreg.score(df_valid_features, df_valid_target), 3))

Лучшая модель логистической регрессии определена со следующими параметрами: {'solver': 'newton-cg', 'penalty': 'l2', 'max_iter': 40}
Достоверность модели логистической регрессии на валидационной выборке составила 0.76


По полученным результатам видно, что при валидации все модели удовлетворяют заданному критерию достоверности - 0.75.
При этом достоверность модели логистической регрессии находится на границе допустимого значения, а значит, при тестовой проверке может перестать ему удовлетворять.

Модель случайного леса обладает лучшей достоверностью, в связи с чем, будем считать ее основной. На следующем шаге проведем ее проверку на тестовой выборке. 

# Проверка модели

Проверим достоверность модели случайного леса на тестовой выборке.

In [ ]:
model_rf.score(df_test_features, df_test_target)

0.8227060653188181

Полученное значение на 0,4% лучше полученного на валидационной выборке, что объясняется несбалансированностью исходных данных для обучения модели.

# Проверка эффективности модели

Проверим адекватность полученного значения достоверности модели. 
Для этого при помощи функции `DummyClassifier` найдем аналогичное значение для модели, принимающей на вход наиболее часто встречающееся значение в обучающей выборке `df_train_target` - "0".

In [ ]:
DummyClassifier(strategy="most_frequent").fit(df_train_features, df_train_target).score(df_train_features, df_train_target)

0.6934647302904564

Полученное значение означает, что в случае предложения всем пользователям тарифа "Смарт", то вероятность успеха составит 69,3%.
Применение разработанной модели позволит предлагать еще и тариф "Ультра" и, тем самым, увеличить указанный показатель на 12,9%.

# Выводы

В результате выполненной работы получены данные о поведении клиентов тарифов «Смарт» и «Ультра», изучена их структура и проведено разделение на обучающую, валидационную и тестовую выборки.

Лучшее значения показателя достоверности (accuracy) на валидационной выборке составило 81,8% и получено на модели случайного леса.

Проверка указанной модели на тестовой выборке показала увеличение достоверности до уровня 82,2% что объясняется несбалансированностью исходных данных для обучения модели и удовлетворяет предъявленному требованию - 75%. 

Таким образом, полученная модель способна проанализировать поведение клиентов и предложить им новый тариф: «Смарт» или «Ультра».
Эффективность модели заключается в увеличении на 12,9% вероятности успешного предложения тарифа по сравнению со случаем предложения тарифа "Смарт" всем пользователям без исключения.